In [5]:
import pandas as pd

In [9]:
tokenized_data = pd.read_csv('tokenized_data.csv', encoding='utf-8')
tokenized_data.dropna(how='any', axis='index', inplace=True)

In [10]:
tokenized_data.head()

,uid,mid,time,forward_count,comment_count,like_count,tokenized_data
0,fa13974743d3fe6ff40d21b872325e9e,8169f1d45051e08ef213bf1106b1225d,2015-02-14 12:49:58,0.0,0.0,0.0,挣钱 是 一 种 能力 抢 红包 拼 的 是 技术 我 抢 到 了 丁辰灵 和 一起 发出 ...
1,da534fe87e7a52777bee5c30573ed5fd,68cd0258c31c2c525f94febea2d9523b,2015-03-31 13:58:06,0.0,0.0,0.0,淘宝网 这些 傻 逼气 的 劳资 有 火 没 地 儿发尼玛 你们 都 瞎 了
2,e06a22b7e065e559a1f0bf7841a85c51,00b9f86b4915aedb7db943c54fd19d59,2015-06-11 20:39:57,0.0,4.0,3.0,看点 不 能 说 的 你们 都 懂
3,f9828598f9664d4e347ef2048ce17734,c7f6f66044c0c5a3330e2c5371be6824,2015-03-10 18:02:38,0.0,0.0,0.0,多 张
4,d80f3d3c5c1d658e82b837a4dd1af849,bfc0819b83ec59ce767287077f2b3507,2015-02-13 01:09:41,0.0,0.0,0.0,有生之年 我 最 喜欢 的 主跟 我 的 三体勾 搭 到 一起 了 幸福感 爆棚


In [11]:
tokenized_data['tokenized_data'].values

array(['挣钱 是 一 种 能力 抢 红包 拼 的 是 技术 我 抢 到 了 丁辰灵 和 一起 发出 的 现金 红包 幸福感 爆棚 情人节 一起 来 和 粉丝 红包 约 个 会 吧 ￣￣ )',
       '淘宝网 这些 傻 逼气 的 劳资 有 火 没 地 儿发尼玛 你们 都 瞎 了', '看点 不 能 说 的 你们 都 懂', ...,
       '金鱼 的 记忆 只 有 七 秒 股民 也 是',
       '小年 抢 个 好 彩头 我 在 五岳 散人 的 红包 中 抽 到 了 提供 的 “ 快 的 打车元 红包 ” 羊年 未 到 好运 先 来 你 也 快 来 试试 手气',
       '电子 土拨鼠 | 当 虚拟 撞 上 现实 技术 宅 逆袭 浪漫 求婚 今年月 里 的 一 次 求婚 相信 你 肯定 会 记得 ： 汪峰 在 章子怡 岁 生日 宴 上 通过 一 台 托载 着重 达 克拉 英国 钻戒 的 无人机 抱得 了 美人 归并 成功 的 上 了 头 条 本期 栏目 就 让 我们 从 一个 技术 宅 的 角度 一步步 将 虚拟 现实 “ 大卸八块 ”'],
      dtype=object)

In [35]:
from sklearn import model_selection

In [41]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [13]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 200
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)


In [15]:
tokenizer.fit_on_texts(tokenized_data['tokenized_data'].values)

In [16]:
word_index = tokenizer.word_index
print(len(word_index))

645798


In [19]:
X = tokenizer.texts_to_sequences(tokenized_data['tokenized_data'].values)

In [21]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

In [22]:
X.shape

(1154444, 200)

In [29]:
Y = tokenized_data[['forward_count', 'comment_count', 'like_count']].to_numpy()

In [30]:
Y.shape

(1154444, 3)

Train/Test Split

In [36]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X,Y, test_size = 0.1, random_state = 42)

In [38]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(1038999, 200) (1038999, 3)
(115445, 200) (115445, 3)


Train model

In [ ]:
model = keras.Sequential()
model.add(layers.Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(layers.SpatialDropout1D(0.2))
model.add(layers.LSTM(1000, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
   19/14611 [..............................] - ETA: 45:49:31 - loss: 8.9188 - accuracy: 0.3873